# FINM 36700 Homework 7

## Andrew Arbitman, Arush Guliani, Tiago Mambrim Flora, Colin Yao

In [125]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
def load_files(LOADFILE, sheet_name):
  df = pd.read_excel(LOADFILE, sheet_name=sheet_name, index_col = 0)
  return df

def mdd_timeseries(rets):
    cum_rets = (1 + rets).cumprod()
    rolling_max = cum_rets.cummax()
    drawdown = (cum_rets - rolling_max) / rolling_max
    return drawdown

In [127]:
signals = load_files('gmo_analysis_data.xlsx', 'signals')
risk_free_rate = load_files('gmo_analysis_data.xlsx', 'risk-free rate')
total_returns = load_files('gmo_analysis_data.xlsx', 'total returns')

In [128]:
FREQ = 12

In [129]:
excess_returns = total_returns.sub((risk_free_rate['TBill 3M'] / FREQ), axis=0)

### 2. Analyzing GMO

#### 2.1 Performance (GMWAX)

Compute mean, volatility, and Sharpe ratio for GMWAX over three samples:

- inception → 2011

- 2012 → present

- inception → present

Has the mean, vol, and Sharpe changed much since the case?

In [130]:
gmwax_inception_2011 = excess_returns.loc[:'2011-12-31', 'GMWAX']
gmwax_2012_present = excess_returns.loc['2012-01-01':, 'GMWAX']
gmwax_inception_present = excess_returns['GMWAX']

In [131]:
gmwax_inception_2011_mean = gmwax_inception_2011.mean() * FREQ
gmwax_2012_present_mean = gmwax_2012_present.mean() * FREQ
gmwax_inception_present_mean = gmwax_inception_present.mean() * FREQ

gmwax_inception_2011_vol = gmwax_inception_2011.std() * np.sqrt(FREQ)
gmwax_2012_present_vol = gmwax_2012_present.std() * np.sqrt(FREQ)
gmwax_inception_present_vol = gmwax_inception_present.std() * np.sqrt(FREQ)

gmwax_inception_2011_sharpe = gmwax_inception_2011_mean / gmwax_inception_2011_vol * np.sqrt(FREQ)
gmwax_2012_present_sharpe = gmwax_2012_present_mean / gmwax_2012_present_vol * np.sqrt(FREQ)
gmwax_inception_present_sharpe = gmwax_inception_present_mean / gmwax_inception_present_vol * np.sqrt(FREQ)

In [132]:
gmwax_summary_stats = pd.DataFrame({
    'Mean': [gmwax_inception_2011_mean, gmwax_2012_present_mean, gmwax_inception_present_mean],
    'Volatility': [gmwax_inception_2011_vol, gmwax_2012_present_vol, gmwax_inception_present_vol],
    'Sharpe Ratio': [gmwax_inception_2011_sharpe, gmwax_2012_present_sharpe, gmwax_inception_present_sharpe]
}, index=['Inception - 2011', '2012 - Present', 'Inception - Present'])
gmwax_summary_stats

,Mean,Volatility,Sharpe Ratio
Inception - 2011,0.046422,0.110499,1.455305
2012 - Present,0.049157,0.092661,1.837715
Inception - Present,0.047730,0.102209,1.617687


The summary statistics illustrated above show that the mean, volatility, and Sharpe have not changed much since the time of the case.

#### 2.2 Tail Risk (GMWAX)

For all three samples, analyze extreme scenarios:

- minimum return

- 5th percentile (VaR‑5th)

- maximum drawdown (compute on total returns, not excess returns)

(a) Does GMWAX have high or low tail‑risk as seen by these stats?

(b) Does that vary much across the two subsamples?

In [133]:
gmwax_inception_2011_min = gmwax_inception_2011.min()
gmwax_2012_present_min = gmwax_2012_present.min()
gmwax_inception_present_min = gmwax_inception_present.min()

gmwax_inception_2011_var_5th = gmwax_inception_2011.quantile(0.05)
gmwax_2012_present_var_5th = gmwax_2012_present.quantile(0.05)
gmwax_inception_present_var_5th = gmwax_inception_present.quantile(0.05)

#Calculate MDD on total returns, not excess returns
gmwax_inception_2011_mdd = mdd_timeseries(total_returns.loc[:'2011-12-31', 'GMWAX']).min()
gmwax_2012_present_mdd = mdd_timeseries(total_returns.loc['2012-01-01':, 'GMWAX']).min()
gmwax_inception_present_mdd = mdd_timeseries(total_returns['GMWAX']).min()

In [134]:
gmwax_risk_stats = pd.DataFrame({
    'Minimum Return': [gmwax_inception_2011_min, gmwax_2012_present_min, gmwax_inception_present_min],
    '5th Percentile': [gmwax_inception_2011_var_5th, gmwax_2012_present_var_5th, gmwax_inception_present_var_5th],
    'Maximum Drawdown': [gmwax_inception_2011_mdd, gmwax_2012_present_mdd, gmwax_inception_present_mdd]
}, index=['Inception - 2011', '2012 - Present', 'Inception - Present'])
gmwax_risk_stats

,Minimum Return,5th Percentile,Maximum Drawdown
Inception - 2011,-0.149150,-0.044003,-0.293614
2012 - Present,-0.115018,-0.039814,-0.216795
Inception - Present,-0.149150,-0.041147,-0.293614


As seen by these stats, GMWAX has high tail risk, with minimum returns going as low as -14.9% for a one month period.

The risk stats do not vary too greatly across the two subsamples, but the inception - 2011 subsample has higher tail risk.

#### 2.3 Market Exposure (GMWAX)

For all three samples, regress excess returns of GMWAX on excess returns of SPY:

- report estimated alpha, beta, and R²

- is GMWAX a low‑beta strategy? Has that changed since the case?

- does GMWAX provide alpha? Has that changed across subsamples?

In [135]:
#Subsample 1: Inception - 2011
y = excess_returns.loc[:'2011-12-31', 'GMWAX']
X = excess_returns.loc[:'2011-12-31', 'SPY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
alpha_gmwax_subsample1 = model.intercept_ * FREQ
beta_gmwax_subsample1 = model.coef_[0]
r_squared_gmwax_subsample1 = model.score(X, y)
print(f"Inception - 2011:")
print("------------------------")
print(f"Estimated alpha: {alpha_gmwax_subsample1:.4f}")
print(f"Estimated beta: {beta_gmwax_subsample1:.4f}")
print(f"R-squared: {r_squared_gmwax_subsample1:.4f}")

Inception - 2011:
------------------------
Estimated alpha: 0.0270
Estimated beta: 0.5421
R-squared: 0.6487


In [136]:
#Subsample 2: 2012 - Present
y = excess_returns.loc['2012-01-01':, 'GMWAX']
X = excess_returns.loc['2012-01-01':, 'SPY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
alpha_gmwax_subsample2 = model.intercept_ * FREQ
beta_gmwax_subsample2 = model.coef_[0]
r_squared_gmwax_subsample2 = model.score(X, y)
print(f"2012 - Present:")
print("------------------------")
print(f"Estimated alpha: {alpha_gmwax_subsample2:.4f}")
print(f"Estimated beta: {beta_gmwax_subsample2:.4f}")
print(f"R-squared: {r_squared_gmwax_subsample2:.4f}")

2012 - Present:
------------------------
Estimated alpha: -0.0274
Estimated beta: 0.5669
R-squared: 0.7309


In [137]:
#Full Sample: Inception - Present
y = excess_returns['GMWAX']
X = excess_returns['SPY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
alpha_gmwax = model.intercept_ * FREQ
beta_gmwax = model.coef_[0]
r_squared_gmwax = model.score(X, y)
print(f"Inception - Present:")
print("------------------------")
print(f"Estimated alpha: {alpha_gmwax:.4f}")
print(f"Estimated beta: {beta_gmwax:.4f}")
print(f"R-squared: {r_squared_gmwax:.4f}")

Inception - Present:
------------------------
Estimated alpha: 0.0022
Estimated beta: 0.5475
R-squared: 0.6752


Yes, GMWAX is a low-beta strategy. Across the entire sample, the beta is 0.55, which is considered low-beta. The beta has not changed significantly since the case.

Across the entire sample, GMWAX does not provide much alpha. However, that has significantly changed across subsamples. In the inception - 2011 subsample, the annualized alpha was 2.70%, which is significant. However, in the 2012 - present subsample, the annualized alpha was -2.74%, which is significantly negative. This indicates that the GMWAX's ability to generate alpha has deteriorated since the case period.

#### 2.4 Compare to GMGEX

Repeat items 1–3 for GMGEX. What are key differences between the two strategies?

In [138]:
gmgex_inception_2011 = excess_returns.loc[:'2011-12-31', 'GMGEX']
gmgex_2012_present = excess_returns.loc['2012-01-01':, 'GMGEX']
gmgex_inception_present = excess_returns['GMGEX']

In [139]:
gmgex_inception_2011_mean = gmgex_inception_2011.mean() * FREQ
gmgex_2012_present_mean = gmgex_2012_present.mean() * FREQ
gmgex_inception_present_mean = gmgex_inception_present.mean() * FREQ

gmgex_inception_2011_vol = gmgex_inception_2011.std() * np.sqrt(FREQ)
gmgex_2012_present_vol = gmgex_2012_present.std() * np.sqrt(FREQ)
gmgex_inception_present_vol = gmgex_inception_present.std() * np.sqrt(FREQ)

gmgex_inception_2011_sharpe = gmgex_inception_2011_mean / gmgex_inception_2011_vol * np.sqrt(FREQ)
gmgex_2012_present_sharpe = gmgex_2012_present_mean / gmgex_2012_present_vol * np.sqrt(FREQ)
gmgex_inception_present_sharpe = gmgex_inception_present_mean / gmgex_inception_present_vol * np.sqrt(FREQ)

In [140]:
gmgex_summary_stats = pd.DataFrame({
    'Mean': [gmgex_inception_2011_mean, gmgex_2012_present_mean, gmgex_inception_present_mean],
    'Volatility': [gmgex_inception_2011_vol, gmgex_2012_present_vol, gmgex_inception_present_vol],
    'Sharpe Ratio': [gmgex_inception_2011_sharpe, gmgex_2012_present_sharpe, gmgex_inception_present_sharpe]
}, index=['Inception - 2011', '2012 - Present', 'Inception - Present'])
gmgex_summary_stats

,Mean,Volatility,Sharpe Ratio
Inception - 2011,-0.003823,0.147253,-0.089938
2012 - Present,0.013182,0.228077,0.200206
Inception - Present,0.004312,0.189982,0.078619


Yes, the mean, volatility, and Sharpe have changed much since the case. In the subsample before the case, the mean and Sharpe were both negative. However, in the subsample after the case, the mean and Sharpe are both positive.

In [141]:
gmgex_inception_2011_min = gmgex_inception_2011.min()
gmgex_2012_present_min = gmgex_2012_present.min()
gmgex_inception_present_min = gmgex_inception_present.min()

gmgex_inception_2011_var_5th = gmgex_inception_2011.quantile(0.05)
gmgex_2012_present_var_5th = gmgex_2012_present.quantile(0.05)
gmgex_inception_present_var_5th = gmgex_inception_present.quantile(0.05)

#Calculate MDD on total returns, not excess returns
gmgex_inception_2011_mdd = mdd_timeseries(total_returns.loc[:'2011-12-31', 'GMGEX']).min()
gmgex_2012_present_mdd = mdd_timeseries(total_returns.loc['2012-01-01':, 'GMGEX']).min()
gmgex_inception_present_mdd = mdd_timeseries(total_returns['GMGEX']).min()

In [142]:
gmgex_risk_stats = pd.DataFrame({
    'Minimum Return': [gmgex_inception_2011_min, gmgex_2012_present_min, gmgex_inception_present_min],
    '5th Percentile': [gmgex_inception_2011_var_5th, gmgex_2012_present_var_5th, gmgex_inception_present_var_5th],
    'Maximum Drawdown': [gmgex_inception_2011_mdd, gmgex_2012_present_mdd, gmgex_inception_present_mdd]
}, index=['Inception - 2011', '2012 - Present', 'Inception - Present'])
gmgex_risk_stats

,Minimum Return,5th Percentile,Maximum Drawdown
Inception - 2011,-0.151592,-0.082292,-0.555630
2012 - Present,-0.658863,-0.065603,-0.737364
Inception - Present,-0.658863,-0.075737,-0.761812


As seen by these stats, GMWAX has extremely high tail risk, with minimum returns going as low as -65.9% for a one month period, as well as maximum drawdown being as low as -76.2%.

The risk stats vary greatly across the two subsamples. In the subsample before the case, the minimum return was -15.2% while in the subsample after the case, the minimum return was -65.9%.

In [143]:
#Subsample 1: Inception - 2011
y = excess_returns.loc[:'2011-12-31', 'GMGEX']
X = excess_returns.loc[:'2011-12-31', 'SPY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
alpha_gmgex_subsample1 = model.intercept_ * FREQ
beta_gmgex_subsample1 = model.coef_[0]
r_squared_gmgex_subsample1 = model.score(X, y)
print(f"Inception - 2011:")
print("------------------------")
print(f"Estimated alpha: {alpha_gmgex_subsample1:.4f}")
print(f"Estimated beta: {beta_gmgex_subsample1:.4f}")
print(f"R-squared: {r_squared_gmgex_subsample1:.4f}")

Inception - 2011:
------------------------
Estimated alpha: -0.0312
Estimated beta: 0.7642
R-squared: 0.7259


In [144]:
#Subsample 2: 2012 - Present
y = excess_returns.loc['2012-01-01':, 'GMGEX']
X = excess_returns.loc['2012-01-01':, 'SPY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
alpha_gmgex_subsample2 = model.intercept_ * FREQ
beta_gmgex_subsample2 = model.coef_[0]
r_squared_gmgex_subsample2 = model.score(X, y)
print(f"2012 - Present:")
print("------------------------")
print(f"Estimated alpha: {alpha_gmgex_subsample2:.4f}")
print(f"Estimated beta: {beta_gmgex_subsample2:.4f}")
print(f"R-squared: {r_squared_gmgex_subsample2:.4f}")

2012 - Present:
------------------------
Estimated alpha: -0.0977
Estimated beta: 0.8213
R-squared: 0.2532


In [145]:
#Full Sample: Inception - Present
y = excess_returns['GMGEX']
X = excess_returns['SPY'].values.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
alpha_gmgex = model.intercept_ * FREQ
beta_gmgex = model.coef_[0]
r_squared_gmgex = model.score(X, y)
print(f"Inception - Present:")
print("------------------------")
print(f"Estimated alpha: {alpha_gmgex:.4f}")
print(f"Estimated beta: {beta_gmgex:.4f}")
print(f"R-squared: {r_squared_gmgex:.4f}")

Inception - Present:
------------------------
Estimated alpha: -0.0608
Estimated beta: 0.7816
R-squared: 0.3984


Yes, GMGEX is a low-beta strategy. Across the entire sample, the beta is 0.78, which is considered low-beta. The beta has changed since the case, from 0.76 before the case to 0.82 after the case.

Across the entire sample, GMGEX provides negative alpha. The alpha has changed since the case, from -0.03 before the case to -0.10 after the case.

The key differences between the GMGEX and GMWAX strategies can be observed in their risk and return profiles: 

- Historically, GMWAX (as shown by the statistics) exhibited higher annualized returns and Sharpe Ratios than GMGEX. This means GMWAX provided better risk-adjusted returns over various periods.

- GMWAX tends to have lower volatility compared to GMGEX. This indicates that GMWAX's returns are less variable and the fund takes on less absolute risk.

- GMWAX also experienced smaller maximum drawdowns and less severe negative return periods than GMGEX, suggesting better downside protection.

In summary, while both are alternative strategies focused on risk-managed equity exposures, GMWAX has generally provided higher returns with lower risk and better downside protection compared to GMGEX.

### 3. Forecast Regressions

#### 3.1 Lagged Regression

Consider the regression with predictors lagged one period
 
Estimate the regression and report the R^2, as well as the OLS estimates for alpha and beta. Do this for:

- X as a single regressor, the dividend–price ratio (DP)

- X as a single regressor, the earnings–price ratio (EP)

- X with three regressors: DP, EP, and the 10‑year yield

For each, report the R^2.

In [187]:
#Single Regressor: D/P
# Ensure predictors at t-1 are predicting return at t

X = signals[['SPX D/P']].shift(1)
y = excess_returns['SPY']
# Align X and y so all rows are matched on time, and remove nulls due to shift
data = pd.concat([y, X], axis=1).dropna()
y_aligned = data['SPY']
X_aligned = data[['SPX D/P']]

model_dp = LinearRegression()
model_dp.fit(X_aligned, y_aligned)
alpha_dp = model_dp.intercept_ * FREQ
beta_dp = model_dp.coef_[0]
r_squared_dp = model_dp.score(X_aligned, y_aligned)

print(f"D/P Regression:")
print("------------------")
print(f"Estimated alpha: {alpha_dp:.4f}")
print(f"Estimated beta: {beta_dp:.4f}")
print(f"R-squared: {r_squared_dp:.4f}")

D/P Regression:
------------------
Estimated alpha: -0.1681
Estimated beta: 1.1718
R-squared: 0.0116


In [188]:
#Single Regressor: E/P
y = excess_returns['SPY']
X = signals[['SPX E/P']].shift(1)
data = pd.concat([y, X], axis=1).dropna()
y_aligned = data['SPY']
X_aligned = data[['SPX E/P']]

model_ep = LinearRegression()
model_ep.fit(X_aligned, y_aligned)
alpha_ep = model_ep.intercept_ * FREQ
beta_ep = model_ep.coef_[0]
r_squared_ep = model_ep.score(X_aligned, y_aligned)

print(f"E/P Regression:")
print("------------------")
print(f"Estimated alpha: {alpha_ep:.4f}")
print(f"Estimated beta: {beta_ep:.4f}")
print(f"R-squared: {r_squared_ep:.4f}")

E/P Regression:
------------------
Estimated alpha: -0.0863
Estimated beta: 0.2640
R-squared: 0.0058


In [191]:
#Three Regressors: E/P, D/P, 10-Year Yield
y = excess_returns['SPY']
X = signals[['SPX E/P', 'SPX D/P', 'T-Note 10YR']].shift(1)
data = pd.concat([y, X], axis=1).dropna()
y_aligned = data['SPY']
X_aligned = data[['SPX E/P', 'SPX D/P', 'T-Note 10YR']]

model_dp_ep_yield = LinearRegression()
model_dp_ep_yield.fit(X_aligned, y_aligned)
alpha_dp_ep_yield = model_dp_ep_yield.intercept_ * FREQ
beta1_dp_ep_yield = model_dp_ep_yield.coef_[0]
beta2_dp_ep_yield = model_dp_ep_yield.coef_[1]
beta3_dp_ep_yield = model_dp_ep_yield.coef_[2]
r_squared_dp_ep_yield = model_dp_ep_yield.score(X_aligned, y_aligned)

print(f"D/P, E/P, and 10-Year Yield Regression:")
print("--------------------------------------")
print(f"Estimated alpha: {alpha_dp_ep_yield:.4f}")
print(f"Estimated D/P beta: {beta1_dp_ep_yield:.4f}")
print(f"Estimated E/P beta: {beta2_dp_ep_yield:.4f}")
print(f"Estimated 10-Year Yield beta: {beta3_dp_ep_yield:.4f}")
print(f"R-squared: {r_squared_dp_ep_yield:.4f}")

D/P, E/P, and 10-Year Yield Regression:
--------------------------------------
Estimated alpha: -0.0425
Estimated D/P beta: 0.1368
Estimated E/P beta: 0.5689
Estimated 10-Year Yield beta: -0.1978
R-squared: 0.0145


#### 3.2 Trading Strategy from Forecasts

For each of the three regressions:

- Build the forecasted SPY return

- Set the scale (portfolio weight) to w_t = 100 * (forecasted SPY return)_t+1.


For each strategy, compute:

- mean, volatility, Sharpe

- max drawdown

- market alpha

- market beta

- market information ratio

In [165]:
def compute_stats(strategy_returns, market_excess):
    mean = strategy_returns.mean() * FREQ
    vol = strategy_returns.std() * np.sqrt(FREQ)
    sharpe = mean / vol
    mdd = mdd_timeseries(strategy_returns)
    mkt = market_excess
    X = mkt.values.reshape(-1, 1)
    y = strategy_returns.values
    lr = LinearRegression().fit(X, y)
    alpha = lr.intercept_ * FREQ
    beta = lr.coef_[0]
    residuals = y - lr.predict(X)
    tracking_error = np.std(residuals) * np.sqrt(FREQ)
    info_ratio = alpha / tracking_error
    return {
        'Mean': mean,
        'Volatility': vol,
        'Sharpe Ratio': sharpe,
        'Max Drawdown': mdd.min(),
        'Alpha': alpha,
        'Beta': beta,
        'Information Ratio': info_ratio
    }

In [ ]:
strategies_results = {}

# 1. D/P regression
coeff_dp = model_dp.coef_[0]
alpha_dp = model_dp.intercept_
dp_forecast = alpha_dp + coeff_dp * signals['SPX D/P']
dp_forecast_aligned = dp_forecast.iloc[:-1].values  
dp_returns_aligned = excess_returns['SPY'].iloc[1:].values  
w_dp = 100 * dp_forecast_aligned
dp_strategy_ret = w_dp * dp_returns_aligned
dp_strategy_ret_series = pd.Series(dp_strategy_ret, index=excess_returns['SPY'].iloc[1:].index)
dp_market_series = excess_returns['SPY'].iloc[1:]
strategies_results['D/P'] = compute_stats(dp_strategy_ret_series, dp_market_series)

# 2. E/P regression
coeff_ep = model_ep.coef_[0]
alpha_ep = model_ep.intercept_
ep_forecast = alpha_ep + coeff_ep * signals['SPX E/P']
ep_forecast_aligned = ep_forecast.iloc[:-1].values
ep_returns_aligned = excess_returns['SPY'].iloc[1:].values
w_ep = 100 * ep_forecast_aligned
ep_strategy_ret = w_ep * ep_returns_aligned
ep_strategy_ret_series = pd.Series(ep_strategy_ret, index=excess_returns['SPY'].iloc[1:].index)
ep_market_series = excess_returns['SPY'].iloc[1:]
strategies_results['E/P'] = compute_stats(ep_strategy_ret_series, ep_market_series)

# 3. D/P, E/P, 10Y regression
coefs = model_dp_ep_yield.coef_
alpha_multi = model_dp_ep_yield.intercept_
multi_forecast = (
    alpha_multi
    + coefs[0] * signals['SPX E/P']
    + coefs[1] * signals['SPX D/P']
    + coefs[2] * signals['T-Note 10YR']
)
multi_forecast_aligned = multi_forecast.iloc[:-1].values
multi_returns_aligned = excess_returns['SPY'].iloc[1:].values
w_multi = 100 * multi_forecast_aligned
multi_strategy_ret = w_multi * multi_returns_aligned
multi_strategy_ret_series = pd.Series(multi_strategy_ret, index=excess_returns['SPY'].iloc[1:].index)
multi_market_series = excess_returns['SPY'].iloc[1:]
strategies_results['D/P, E/P, T-Note'] = compute_stats(multi_strategy_ret_series, multi_market_series)

results_df = pd.DataFrame(strategies_results).T[['Mean','Volatility','Sharpe Ratio','Max Drawdown','Alpha','Beta','Information Ratio']]
results_df

,Mean,Volatility,Sharpe Ratio,Max Drawdown,Alpha,Beta,Information Ratio
D/P,0.086599,0.158435,0.546594,-0.707442,0.018636,0.804736,0.188125
E/P,0.073108,0.131976,0.553950,-0.579859,0.008299,0.767394,0.139652
"D/P, E/P, T-Note",0.093602,0.156400,0.598479,-0.645193,0.027164,0.786678,0.273700


#### 3.3 Risk Characteristics

- For both strategies, the market, and GMO, compute monthly VaR at 0.05 (use the historical quantile).

- The case mentions stocks under‑performed short‑term bonds from 2000–2011. Does the dynamic portfolio above under‑perform the risk‑free rate over this time?

- Based on the regression estimates, in how many periods do we estimate a negative risk premium?

- Do you believe the dynamic strategy takes on extra risk?

In [200]:
def compute_var(series, alpha=0.05):
    # Ensure input is a pandas Series for quantile method
    if isinstance(series, (pd.Series, pd.DataFrame)):
        return series.quantile(alpha)
    else:
        # Convert numpy array to pandas Series
        return pd.Series(series).quantile(alpha)

# Prepare strategies for VaR evaluation
strategies_for_var = {
    'Market': excess_returns['SPY'],
    'GMO': excess_returns['GMGEX'],
    'D/P': dp_strategy_ret_series,
    'E/P': ep_strategy_ret_series,
    'D/P, E/P, T-Note': multi_strategy_ret_series
}

var_results = {}
for name, returns in strategies_for_var.items():
    var_results[name] = compute_var(returns, 0.05)

var_results_df = pd.DataFrame.from_dict(var_results, orient='index', columns=['VaR 5%'])
var_results_df

,VaR 5%
Market,-0.078272
GMO,-0.075737
D/P,-0.050837
E/P,-0.048324
"D/P, E/P, T-Note",-0.052337


In [207]:
# Convert the annualized risk-free rate to the period rate, then take the relevant time slice
rf_periodic = risk_free_rate['TBill 3M'] / FREQ
rf_periodic = pd.Series(rf_periodic, index=risk_free_rate.index)

rf_2000_2011 = rf_periodic.loc['2000-01-01':'2011-12-31']
dynamic_2000_2011 = dp_strategy_ret_series.loc['2000-01-01':'2011-12-31']
risk_free_mean = rf_2000_2011.mean() * FREQ
dynamic_mean = dynamic_2000_2011.mean() * FREQ
print(f"Risk-free rate mean: {risk_free_mean:.4f}")
print(f"Dynamic strategy mean: {dynamic_mean:.4f}")

Risk-free rate mean: 0.0229
Dynamic strategy mean: 0.0451


Yes, the dynamic portfolio underperforms the risk-free rate over this time period. 

In [171]:
neg_rp_count = (dp_forecast_ret < 0).sum()
print(f"Periods with negative risk premium forecast (D/P regression): {neg_rp_count}")

Periods with negative risk premium forecast (D/P regression): 13


Yes, the dynamic strategy appears to take on extra risk. This is evident from its higher volatility and larger drawdowns compared to the benchmark and other static strategies, as shown in the risk and performance tables above. The higher risk may be due to the timing approach or increased exposure to risky assets during certain periods. While it can potentially offer higher returns or risk-adjusted performance, it does so by assuming greater downside risk.


### 4. Out-of-Sample Forecasting

Focus on using both DP and EP as signals. Compute out‑of‑sample (OOS) statistics:

Procedure (Rolling OOS):

- Start at t = 60.

- Estimate the lagged regression using data through time t.

- Using the estimated parameters and x_t, compute the forecast for t+1

- Forecast error: 

- Move to t = 61 and iterate.

Also compute the null forecast and errors.

1. Report the Out-of-Sample R^2

2. Redo 3.2 with OOS forecasts. How does the OOS strategy compare to the in‑sample version of 3.2?

3. Redo 3.3 with OOS forecasts. Is the point‑in‑time version of the strategy riskier?

In [ ]:
start = 60
X_signals = signals[['SPX D/P', 'SPX E/P']]
y = excess_returns['SPY']

oos_forecast = []
oos_actual = []
oos_null = []

for t in range(start, len(X_signals) - 1):
    X_train = X_signals.iloc[0:t]
    y_train = y.iloc[1:t+1]
    
    valid_idx = X_train.index.intersection(y_train.index)
    X_train = X_train.loc[valid_idx]
    y_train = y_train.loc[valid_idx]
    
    X_pred = X_signals.iloc[[t]]
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    forecast = lr.predict(X_pred)[0]
    
    actual = y.iloc[t+1]
    
    null_forecast = y_train.mean()
    
    oos_forecast.append(forecast)
    oos_actual.append(actual)
    oos_null.append(null_forecast)

oos_forecast = np.array(oos_forecast)
oos_actual = np.array(oos_actual)
oos_null = np.array(oos_null)

sspe_forecast = np.sum((oos_actual - oos_forecast)**2)
sspe_null = np.sum((oos_actual - oos_null)**2)
oos_r2 = 1 - sspe_forecast / sspe_null
print(f"Out-of-Sample R^2: {oos_r2:.4f}")

Out-of-Sample R^2: -0.0740


In [ ]:
oos_weights = 100 * oos_forecast
oos_strategy_returns = oos_weights * oos_actual

oos_dates = y.index[start+1:start+1+len(oos_forecast)]
oos_strategy_returns_series = pd.Series(oos_strategy_returns, index=oos_dates)
oos_actual_series = pd.Series(oos_actual, index=oos_dates)

oos_strategy_stats = pd.DataFrame(compute_stats(oos_strategy_returns_series, oos_actual_series), index=['OOS Strategy'])
oos_strategy_stats

,Mean,Volatility,Sharpe Ratio,Max Drawdown,Alpha,Beta,Information Ratio
OOS Strategy,0.007087,0.247899,0.028588,-0.918996,0.023134,-0.182286,0.094047


The OOS strategy has a lower Sharpe ratio and a higher tail risk than what we found in 3.2. Also, the market beta for the OOS strategy is negative while in the full sample, it is positive.

In [186]:
market_oos = oos_actual_series
gmgex_oos = excess_returns['GMGEX'].loc[oos_dates]

strategies_for_var = {
    'OOS Strategy': oos_strategy_returns_series,
    'Market (SPY)': market_oos,
    'GMO (GMGEX)': gmgex_oos
}

var_results = {}
for name, returns in strategies_for_var.items():
    var_results[name] = compute_var(returns, 0.05)

var_results_df = pd.DataFrame.from_dict(var_results, orient='index', columns=['VaR 5%'])
display(var_results_df)

oos_strategy_mean = oos_strategy_returns_series['2000-01-01':'2011-12-31'].mean() * FREQ
risk_free_mean = risk_free_rate['TBill 3M']['2000-01-01':'2011-12-31'].mean() * FREQ
print(f"OOS Strategy mean: {oos_strategy_mean:.4f}")
print(f"Risk-free rate mean: {risk_free_mean:.4f}")

,VaR 5%
OOS Strategy,-0.055903
Market (SPY),-0.074125
GMO (GMGEX),-0.075742


OOS Strategy mean: -0.0773
Risk-free rate mean: 0.2745


The OOS strategy underperforms the risk-free rate over this time period. The OOS strategy is not significantly riskier as the 5% VaR is not changed significantly from the full sample to the OOS.